In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# LAB: Generando nuestra propia API
# Ahora somos el **CLIENTE**

## Introducción e instrucciones

Si todo salió bien, en este preciso momento tienen otra notebook abierta, simulando ser un **SERVIDOR**, en la cual definieron tres funciones asociadas cada una a un endpoint de nuestra API. 
La idea de esta notebook es hacer los distintos llamados a esos endpoints para que la magia de nuestra API suceda. Si todavía NO construyeron los endpoints, entonces vuelvan a la notebook **Clase_PIckle_shelve_flask_API** y sigan las instrucciones.

Importamos las librerías que vamos a utilizar

In [ ]:
# Algunas de las de siempre
import numpy as np
import pandas as pd

# Las que vamos a necesitar para hacer los llamados y para lidiar con los datos en formato json
import requests
from flask import  Flask, request, jsonify, render_template
import json



***

## **PRIMER ENDPOINT:** entrenar el modelo

Para este primer endpoint, necesitamos cargar la base con los datos de los jugadores argentinos. 

In [ ]:
df = pd.read_csv('../Data/fifa_18_jugadores_argentinos_dos.csv',index_col='Unnamed: 0')

Usando la librería **request**, usamos el método **post** para enviar el input del primer llamado (en la notebook que simula ser el servidor está el detalle de lo que tenemos que pasarle a este **endpoint**). Tengan en cuenta que el dataframe lo tienen que pasar a un formato json con el métod de pandas <a class="reference internal" href=https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html><b><code>.to_json()</code></b></a>)

In [ ]:
# Definimos los datos que vamos a pasar en el request
base=df.to_json() #dataframe en formato json
lista_predictores=df.drop(columns=['ID','full_name','club','puntaje_global']).columns.values #features para entrenar el modelo
variable_target='puntaje_global' #variable a predecir
directory='modelos' #dirección donde vamos a guardar el modelo  
name='predictor_jugadores' #nombre modelo
alphas=list(np.linspace(0.001, 10, 10)) #alphas que vamos a usar en el cross-validation con ElasticNet
url = 'http://localhost:5009/entrenar_modelo' #definimos la URL para hacer el llamado local con el endpoint correspondiente

In [ ]:
# Hacemos el pedido y vemos los resultados.
# Si todo salió bien en la carpeta "modelos" se tiene que haber generado un .pkl con el modelo entrenado
modelo = requests.post(url,
                       json = {'base' : base,
                               'lista_predictores':list(lista_predictores),
                               'target':variable_target,
                               'alphas':alphas,
                               'directory':directory,
                               'name':name})
print(modelo.json())

***

## **SEGUNDA FUNCIÓN/ENDOPOINT:** coeficientes del modelo

In [ ]:
#Definimos los datos que vamos a pasar en el pedido
dir_modelo='modelos' #dirección donde se encuentra el modelo entrenado
nombre_modelo='predictor_jugadores' #nombre del modelo entrenado
palette='hot' #paleta de colores para usar en el gráfico de barras
size=(25,8) #tamaño de la figura
predictores=list(lista_predictores) #lista con los features en el mismo orden en el que fueron pasados en el modelo
url = 'http://localhost:5009/plot_coeficientes' #definimos la URL para hacer el llamado local con el endpoint correspondiente

In [ ]:
# Hacemos el pedido e imprimimos el output
coef = requests.post(url,
                     json = {'direccion' : dir_modelo,
                             'nombre_modelo':nombre_modelo,
                             'paleta_colores':palette,
                             'tamano':size,
                             'predictores':predictores})
print(coef.json())

***

## **TERCERA FUNCIÓN/ENDOPOINT:** predicciones

In [ ]:
#Definimos las variables que vamos a utilizar para el pedido
direccion_modelo='modelos' #dirección donde se encuentra el modelo grabado
nombre_modelo='predictor_jugadores' #nombre del modelo grabado
nombre_escalador='predictor_jugadores_scaler' #nombre del escalador que entrenamos cuando corrimos el modelo
features=df.drop(columns=['ID','full_name','club','puntaje_global']).iloc[3].to_json() #elegimos features de un jugador 
#cualquiera de la base para chequear que funcione bien el llamado
url = 'http://localhost:5009/prediccion' #definimos la URL para hacer el llamado local con el endpoint correspondiente


In [ ]:
#Hacemos el llamado
pred = requests.get(url, 
                    params = {'direccion':direccion_modelo,
                              'nombre_modelo':nombre_modelo,
                              'nombre_scalador':nombre_escalador,
                              'features':[features]})
print(pred.json())

### ¡Ahí tenemos nuestra predicción! ¡Congrats!
<img src="img/we_did_it.jpg" alt="Drawing" style="width: 300px;"/>